# Trabalho Final - Deep Learning and Reinforcement Learning

### Integrantes:

###    1) Jurandir Ventura - RM336634
###    2) Marcelo Preto - RM336632
###    3) Fabio Rizzi - RM336656
###    4) Thiago Alexandre – RM336583

## Extreme Learning Machines

### Exercício #4 - 1 PONTO

### Utilizando a biblioteca elm  do pacote sklearn_extensions.extreme_learning_machines e o dataset Mushroom Classification (disponível no repositório da disciplina e em https://www.kaggle.com/uciml/mushroom-classification) encontre a configuração de ELM que atinge a melhor acurácia

### Dicas:
#### Utilize o exemplo plot_eml_comparsion.py como base;
#### Não é necessário plotar as fronteiras de decisão do classificador;